In [1]:
from datasets import load_metric
from datasets import load_dataset 
from transformers import AutoImageProcessor
from transformers import AutoModelForImageClassification, TrainingArguments, Trainer
import numpy as np
import torch
from torchvision.transforms import (
    CenterCrop,
    Compose,
    Normalize,
    RandomHorizontalFlip,
    RandomResizedCrop,
    Resize,
    ToTensor,
)


2023-09-15 16:43:06.299583: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-15 16:43:06.962379: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
dataset = load_dataset("imagefolder", data_dir="DATASET_TRAIN/")

model_checkpoint = "google/vit-base-patch16-224-in21k" # pre-trained model from which to fine-tune
#model_checkpoint = "google/vit-large-patch16-224-in21k" # pre-trained model from which to fine-tune

metric = load_metric("accuracy")

labels = dataset["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label

image_processor  = AutoImageProcessor.from_pretrained(model_checkpoint)


print(dataset["train"].features)
print(dataset["validation"].features)
print(dataset)
print(id2label)



Resolving data files:   0%|          | 0/8000 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/2000 [00:00<?, ?it/s]

/tmp/ipykernel_223314/3052539720.py:6: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


{'image': Image(decode=True, id=None), 'label': ClassLabel(names=['bw', 'color', 'ph_tab', 'real', 'replay'], id=None)}
{'image': Image(decode=True, id=None), 'label': ClassLabel(names=['bw', 'color', 'ph_tab', 'real', 'replay'], id=None)}
DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 8000
    })
    validation: Dataset({
        features: ['image', 'label'],
        num_rows: 2000
    })
})
{0: 'bw', 1: 'color', 2: 'ph_tab', 3: 'real', 4: 'replay'}


In [3]:
normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
if "height" in image_processor.size:
    size = (image_processor.size["height"], image_processor.size["width"])
    crop_size = size
    max_size = None
elif "shortest_edge" in image_processor.size:
    size = image_processor.size["shortest_edge"]
    crop_size = (size, size)
    max_size = image_processor.size.get("longest_edge")

train_transforms = Compose(
        [
            RandomResizedCrop(crop_size),
            RandomHorizontalFlip(),
            ToTensor(),
            normalize,
        ]
    )

val_transforms = Compose(
        [
            Resize(size),
            CenterCrop(crop_size),
            ToTensor(),
            normalize,
        ]
    )

def preprocess_train(example_batch):
    """Apply train_transforms across a batch."""
    example_batch["pixel_values"] = [
        train_transforms(image.convert("RGB")) for image in example_batch["image"]
    ]
    return example_batch

def preprocess_val(example_batch):
    """Apply val_transforms across a batch."""
    example_batch["pixel_values"] = [val_transforms(image.convert("RGB")) for image in example_batch["image"]]
    return example_batch

In [4]:
# split up training into training + validation
#splits = dataset["train"].train_test_split(test_size=0.1)
#train_ds = splits['train']
#val_ds = splits['test']

train_ds = dataset['train']
val_ds = dataset['validation']

print(len(train_ds))
print(len(val_ds))

train_ds.set_transform(preprocess_train)
val_ds.set_transform(preprocess_val)



8000
2000


In [5]:
model = AutoModelForImageClassification.from_pretrained(
    model_checkpoint, 
    label2id=label2id,
    id2label=id2label,
    ignore_mismatched_sizes = True, # provide this in case you're planning to fine-tune an already fine-tuned checkpoint
)

batch_size = 8 # batch size for training and evaluation

model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-01_epoch_model",
    remove_unused_columns=False,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
)

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
# the compute_metrics function takes a Named Tuple as input:
# predictions, which are the logits of the model as Numpy arrays,
# and label_ids, which are the ground-truth labels as Numpy arrays.
def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)


def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}


trainer = Trainer(
    model,
    args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
    data_collator=collate_fn,
)

train_results = trainer.train()
# rest is optional but nice to have
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()


metrics = trainer.evaluate()
# some nice to haves:
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

/datadisk/anaconda3/envs/tranf/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 8000
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 4
  Total optimization steps = 250
  Number of trainable parameters = 85802501


Epoch,Training Loss,Validation Loss,Accuracy
1,0.394600,0.409980,0.955500


***** Running Evaluation *****
  Num examples = 2000
  Batch size = 8
Saving model checkpoint to vit-base-patch16-224-in21k-01_epoch_model/checkpoint-250
Configuration saved in vit-base-patch16-224-in21k-01_epoch_model/checkpoint-250/config.json
Model weights saved in vit-base-patch16-224-in21k-01_epoch_model/checkpoint-250/pytorch_model.bin
Image processor saved in vit-base-patch16-224-in21k-01_epoch_model/checkpoint-250/preprocessor_config.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from vit-base-patch16-224-in21k-01_epoch_model/checkpoint-250 (score: 0.9555).
Saving model checkpoint to vit-base-patch16-224-in21k-01_epoch_model
Configuration saved in vit-base-patch16-224-in21k-01_epoch_model/config.json
Model weights saved in vit-base-patch16-224-in21k-01_epoch_model/pytorch_model.bin
Image processor saved in vit-base-patch16-224-in21k-01_epoch_model/preprocessor_config.json
***** Running Evaluation *****
  Num example

***** train metrics *****
  epoch                    =         1.0
  total_flos               = 577375838GF
  train_loss               =       0.818
  train_runtime            =  0:08:26.36
  train_samples_per_second =      15.799
  train_steps_per_second   =       0.494


***** eval metrics *****
  epoch                   =        1.0
  eval_accuracy           =     0.9555
  eval_loss               =       0.41
  eval_runtime            = 0:01:15.35
  eval_samples_per_second =     26.541
  eval_steps_per_second   =      3.318


# INFERENCING

In [1]:
from transformers import AutoModelForImageClassification, AutoImageProcessor
from PIL import Image
import torch
from transformers import pipeline
from os import listdir
from os.path import isfile, join

image_processor = AutoImageProcessor.from_pretrained("/mnt/vit-large-patch32-384-01_epoch_model")
model = AutoModelForImageClassification.from_pretrained("/mnt/vit-large-patch32-384-01_epoch_model")
    
#pipe = pipeline("image-classification", 
#                model=model,
#                feature_extractor=image_processor)    



2023-09-15 13:52:41.211343: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-15 13:52:41.845204: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [1]:
## Test

from transformers import ViTForImageClassification
from transformers import ViTImageProcessor
import torch
from transformers import pipeline
from PIL import Image
from os import listdir
from os.path import isfile, join

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = ViTForImageClassification.from_pretrained('/mnt/vit-large-patch32-384-01_epoch_model')
model.to(device)

image_processor = ViTImageProcessor.from_pretrained('/mnt/vit-large-patch32-384-01_epoch_model')

pipe = pipeline("image-classification", 
                model=model,
                feature_extractor=image_processor,device=0)  

#img='DATASET_TEST/ph_tab/tab_test/IMG_20230825_103149.jpg'
#image=Image.open(img)

#inputs = image_processor(images=image, return_tensors="pt").to(device)
#pixel_values = inputs.pixel_values

#result=pipe(image)
#print(result)

2023-09-15 14:00:00.101452: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-15 14:00:00.748186: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [1]:
## Test #2
from transformers import AutoModelForImageClassification, AutoImageProcessor
import torch
from transformers import pipeline
from PIL import Image
from os import listdir
from os.path import isfile, join

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


model = AutoModelForImageClassification.from_pretrained("vit-base-patch16-224-in21k-01_epoch_model")
    
    
model.to(device)

image_processor = AutoImageProcessor.from_pretrained("vit-base-patch16-224-in21k-01_epoch_model")

pipe = pipeline("image-classification", 
                model=model,
                feature_extractor=image_processor,device=0)  

#img='DATASET_TEST/ph_tab/tab_test/IMG_20230825_103149.jpg'
#image=Image.open(img)

#inputs = image_processor(images=image, return_tensors="pt").to(device)
#pixel_values = inputs.pixel_values

#result=pipe(image)
#print(result)

2023-09-15 16:55:08.557374: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-15 16:55:09.196055: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
from PIL import Image
def inference_sin(image):
    encoding = image_processor(image.convert("RGB"), return_tensors="pt")
    # forward pass
    with torch.no_grad():
        outputs = model(**encoding)
        logits = outputs.logits
    predicted_class_idx = logits.argmax(-1).item()
    print("Predicted class:", model.config.id2label[predicted_class_idx])

img='DATASET_TEST/ph_tab/tab_test/IMG_20230825_103149.jpg'
img=Image.open(img)
inference_sin(img)
    

Predicted class: ph_tab


In [5]:
# single image infrence

img='DATASET_TEST/ph_tab/tab_test/IMG_20230825_103149.jpg'

result=pipe(img)
max_score_dict = max(result, key=lambda x: x['score'])
print(f"{max_score_dict['label']} : {round(max_score_dict['score'],2)}")
#print(result)

RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor

In [2]:
# Infrence on folder


#image_folder='/datadisk/PH_TAB_IMG_DET/test_images_real/'
image_folder='/datadisk/PH_TAB_IMG_DET/TRANSFORMERS_YWO/DATASET_TEST/ph_tab/tab_test/'

all_images = [f for f in listdir(image_folder) if (isfile(join(image_folder, f)))]

real=0
color=0
bw=0
replay=0
ph_tab=0

for img in all_images:
    result=pipe(image_folder+img)
    max_score_dict = max(result, key=lambda x: x['score'])
    print(f"{max_score_dict['label']} : {round(max_score_dict['score'],2)}")
    if max_score_dict['label']=='real':
        real+=1
    elif max_score_dict['label']=='color':
        color+=1
    elif max_score_dict['label']=='bw':
        bw+=1
    elif max_score_dict['label']=='ph_tab':
        ph_tab+=1
    elif max_score_dict['label']=='replay':
        replay+=1
        
print('++++++++++++++++++++++++++++++++++++++++++++++++++++')
print('real :',real)
print('color :',color)
print('bw :',bw)
print('replay :',replay)
print('ph_tab :',ph_tab)

print('total :',real+color+bw+replay+ph_tab)

        

ph_tab : 1.0
ph_tab : 1.0
ph_tab : 0.99
ph_tab : 0.99
ph_tab : 1.0
ph_tab : 1.0
ph_tab : 0.99
ph_tab : 0.99
ph_tab : 1.0
ph_tab : 1.0
ph_tab : 1.0


/datadisk/anaconda3/envs/tranf/lib/python3.8/site-packages/transformers/pipelines/base.py:1045: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


ph_tab : 1.0
ph_tab : 1.0
ph_tab : 0.99
ph_tab : 1.0
ph_tab : 0.99
ph_tab : 1.0
ph_tab : 1.0
ph_tab : 1.0
ph_tab : 1.0
ph_tab : 1.0
ph_tab : 0.99
ph_tab : 1.0
ph_tab : 1.0
ph_tab : 0.99
ph_tab : 0.99
ph_tab : 0.98
ph_tab : 1.0
ph_tab : 1.0
ph_tab : 1.0
ph_tab : 1.0
ph_tab : 1.0
ph_tab : 0.99
ph_tab : 1.0
ph_tab : 1.0
ph_tab : 0.97
ph_tab : 1.0
ph_tab : 1.0
ph_tab : 0.99
ph_tab : 0.96
ph_tab : 1.0
ph_tab : 1.0
ph_tab : 1.0
ph_tab : 1.0
ph_tab : 1.0
ph_tab : 1.0
ph_tab : 1.0
ph_tab : 0.99
ph_tab : 1.0
ph_tab : 1.0
ph_tab : 1.0
ph_tab : 1.0
ph_tab : 0.97
ph_tab : 1.0
ph_tab : 1.0
ph_tab : 0.98
ph_tab : 0.99
ph_tab : 1.0
ph_tab : 0.99
ph_tab : 1.0
ph_tab : 0.99
ph_tab : 0.98
ph_tab : 1.0
ph_tab : 1.0
ph_tab : 1.0
ph_tab : 0.92
ph_tab : 1.0
ph_tab : 1.0
ph_tab : 1.0
ph_tab : 0.96
ph_tab : 1.0
ph_tab : 1.0
ph_tab : 1.0
ph_tab : 1.0
ph_tab : 1.0
ph_tab : 1.0
ph_tab : 1.0
ph_tab : 1.0
ph_tab : 0.99
ph_tab : 1.0
ph_tab : 1.0
ph_tab : 1.0
ph_tab : 1.0
ph_tab : 0.99
ph_tab : 1.0
ph_tab : 1.0
ph_t

In [ ]:
#google/vit-base-patch32-384  -- Testing done
#google/vit-base-patch16-224-in21k -- done 
#google/vit-large-patch16-224-in21k -- done
#google/vit-large-patch32-384-- in progress

#nohup python train.py &

#google/vit-base-patch32-384

#color
real: 44,color: 66,bw: 1,replay: 2,ph_tab: 16
total: 129
accuracy=51%

#bw
real: 0,color: 1,bw: 196,replay: 2,ph_tab: 1
total: 200

#ph
real : 17,color : 4,bw : 0,replay : 9,ph_tab : 70
total : 100

#tab
real : 3,color : 2,bw : 0,replay : 2,ph_tab : 93
total : 100

#replay
real : 8,color : 1,bw : 0,replay : 158,ph_tab : 33
total : 200

#real : 266
color : 52
bw : 0
replay : 4
ph_tab : 78
total : 400

#real : 0
color : 6
bw : 0
replay : 13
ph_tab : 81
total : 100

real : 0
color : 2
bw : 0
replay : 0
ph_tab : 98
total : 100

real : 0
color : 0
bw : 0
replay : 0
ph_tab : 100
total : 100